In [ ]:
!git clone https://github.com/soCzech/TransNet


In [ ]:
pip install pillow


In [ ]:
pip install ffmpeg-python

In [1]:
%cd TransNet

/mnt/f/AIC/TransNet


In [2]:
import ffmpeg
import numpy as np
import tensorflow as tf

from transnet import TransNetParams, TransNet
from transnet_utils import draw_video_with_predictions, scenes_from_predictions

/home/zhentu/anaconda3/envs/tf1.12/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zhentu/anaconda3/envs/tf1.12/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zhentu/anaconda3/envs/tf1.12/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zhentu/anaconda3/envs/tf1.12/lib/python3

In [3]:
import tensorflow as tf
import numpy as np

def conv3d(inp, filters, dilation_rate):
    return tf.keras.layers.Conv3D(filters, kernel_size=3, dilation_rate=(dilation_rate, 1, 1), padding="SAME", activation=tf.nn.relu, use_bias=True, name="Conv3D_{:d}".format(dilation_rate))(inp)

# Create a NumPy array for data1
data1 = np.ones((1, 27, 48, 3))  # Adjust the shape according to your needs

# Add a batch dimension if necessary
data1 = np.expand_dims(data1, axis=0)  # Shape should now be (1, 27, 48, 3)

# Call conv3d with data1
x2 = conv3d(data1, 16, 4)
print(x2.shape)


AttributeError: 'tuple' object has no attribute 'ndims'

In [4]:
params = TransNetParams()
params.CHECKPOINT_PATH = "./model/transnet_model-F16_L3_S2_D256"


In [5]:
net = TransNet(params)

[TransNet] Creating ops.
           Input (?, ?, 27, 48, 3)
           SDDCNN_1
           > DDCNN_1 (?, ?, 27, 48, 64)
           > DDCNN_2 (?, ?, 27, 48, 64)
           MaxPool (?, ?, 13, 24, 64)
           SDDCNN_2
           > DDCNN_1 (?, ?, 13, 24, 128)
           > DDCNN_2 (?, ?, 13, 24, 128)
           MaxPool (?, ?, 6, 12, 128)
           SDDCNN_3
           > DDCNN_1 (?, ?, 6, 12, 256)
           > DDCNN_2 (?, ?, 6, 12, 256)
           MaxPool (?, ?, 3, 6, 256)
           Flatten (?, ?, 4608)
           Dense (?, ?, 256)
           Logits (?, ?, 2)
           Predictions (?, ?)
[TransNet] Network built.
[TransNet] Found 4614850 trainable parameters.
INFO:tensorflow:Restoring parameters from ./model/transnet_model-F16_L3_S2_D256
[TransNet] Parameters restored from 'transnet_model-F16_L3_S2_D256'.


In [6]:
import ffmpeg
video_path = "/mnt/f/AIC/Video/Thời sự toàn cảnh tối 9-7-Nhậu hết rượu, lấy cồn rửa tay ra uống, 4 người nhập viện  VTV24.mp4"
video_stream, err = (
    ffmpeg
    .input(video_path)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(params.INPUT_WIDTH, params.INPUT_HEIGHT))
    .run(capture_stdout=True)
)
video = np.frombuffer(video_stream, np.uint8).reshape([-1, params.INPUT_HEIGHT, params.INPUT_WIDTH, 3])

In [7]:
video.shape

(57629, 27, 48, 3)

In [ ]:
predictions = net.predict_video(video)
predictions

[TransNet] Processing video frames 26150/57629

In [ ]:
draw_video_with_predictions(video, predictions, threshold=0.1)


In [ ]:
scenes = scenes_from_predictions(predictions, threshold=0.1)

In [ ]:
import numpy as np

# Assuming scenes is your array
# Remove the truncation for display
np.set_printoptions(threshold=np.inf)

# Display the array
scenes